Copyright (c) 2020. Cognitive Scale Inc. All rights reserved.
Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/7998b8a481fccd467463deb1fc46d19622079b0e/LICENSE.md)

# Introduction

This fifth notebook shows you how to explore the trust score results in a notebook. If you have not already done so, please run the [fourth notebook](patient-readmission-trust-scan.ipynb) to scan the models for their trust scores.

In this notebook, we will:
1. Load the previously saved trust score reports
2. Convert the results into a dataframe and display them


In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from certifai.scanner.report_reader import ScanReportReader
from certifai.scanner.report_utils import (
    scores, construct_scores_dataframe, construct_feature_scores_dataframe, construct_overall_scores_dataframe,
    construct_group_scores_dataframe)
from certifai.scanner.builder import ExplanationType
from IPython.display import display, Markdown

# Loading the Trust Score Reports

To load the reports, we need to know the use case ID ('readmission') and the scan ID.

List the available use cases, and the scans within the 'readmission' use case.

In [2]:
reader = ScanReportReader("reports")
reader.list_usecases()
scans = reader.list_scans('readmission')
data=[[s['date'], ', '.join(s['reportTypes']), s['id']] for s in scans]
df = pd.DataFrame(data, columns=['date', 'evals', 'scan id']).sort_values(by=['date'], ascending=False)
print(df)

              date                                      evals       scan id
0  20201102T134851  explainability, fairness, atx, robustness  eafa5614d892
1  20201102T131236                                explanation  c0e07952e6d6
2  20201102T103533                                explanation  df7a42f2ab84
3  20201102T102839                                explanation  b6de685a2712
4  20201016T132038                                explanation  5683ba38b998
5  20201011T151543  atx, fairness, robustness, explainability  4b13c55a4131
6  20201011T141903                                explanation  7a2e8b64cbd6


Locate the latest trust score scan and load it.

In [3]:
latest_scan = df[df['evals'].str.contains('fairness') & df['evals'].str.contains('robustness')].iloc[0]
results = reader.load_scan('readmission', latest_scan['scan id'])

# Extract the results

In this section we'll construct dataframes containing the score results and display them.

Print out the overall fairness results.

In [4]:
fairness_scores = list(scores('fairness', results))
display(construct_overall_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,score,score lower bound,score upper bound
0,mlp,fairness,burden,71.720752,67.045924,75.873632
1,logit,fairness,burden,59.551743,53.758901,65.337007


Print out the fairness scores by feature

In [5]:
display(construct_feature_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,age,age lower bound,age upper bound,gender,gender lower bound,gender upper bound,race,race lower bound,race upper bound
0,mlp,fairness,burden,75.494891,70.844715,80.032798,94.906289,87.700702,99.487169,71.953967,67.045924,76.684155
1,logit,fairness,burden,59.686611,53.875415,66.078304,84.501607,73.826148,94.716060,65.890092,59.128323,72.863878


Print out the detailed fairness scores

In [6]:
display(construct_group_scores_dataframe(fairness_scores, include_confidence=True))

,context,eval type,score type,value type,age - 15,age - 15 lower bound,age - 15 upper bound,age - 25,age - 25 lower bound,age - 25 upper bound,...,race - Caucasian upper bound,race - Hispanic,race - Hispanic lower bound,race - Hispanic upper bound,race - Other,race - Other lower bound,race - Other upper bound,race - nan,race - nan lower bound,race - nan upper bound
0,mlp,fairness,burden,burden,0.056508,0.044298,0.069952,0.132073,0.100531,0.167759,...,0.093498,0.080362,0.062368,0.099629,0.064237,0.048607,0.082366,0.017340,0.011066,0.024469
1,logit,fairness,burden,burden,0.031334,0.018616,0.049608,0.218283,0.152212,0.288154,...,0.114462,0.075701,0.051036,0.107286,0.054516,0.032154,0.085820,0.017773,0.009156,0.027991


For an example of visualizing the fairness results in a chart, see the [fairness metrics notebook](https://github.com/CognitiveScale/cortex-certifai-examples/blob/master/notebooks/fairness_metrics/FairnessMetrics.ipynb).

Print out the robustness scores.

In [7]:
df = construct_scores_dataframe(scores('robustness', results), include_confidence=True)
display(df)

,context,robustness,robustness lower bound,robustness upper bound
mlp,mlp,70.003655,65.077138,74.713251
logit,logit,80.816004,76.619526,84.575458


Print out the explainability scores.

In [8]:
df = construct_scores_dataframe(scores('explainability', results), include_confidence=False)
display(df)

,context,explainability,Num features (1),Num features (10),Num features (2),Num features (3),Num features (4),Num features (5),Num features (6),Num features (7),Num features (8),Num features (9)
mlp,mlp,97.265625,75.78125,0.0,23.4375,0.00000,0.78125,0.0,0.0,0.0,0.0,0.0
logit,logit,97.617188,77.34375,0.0,21.8750,0.78125,0.00000,0.0,0.0,0.0,0.0,0.0
